In [1]:
import os
import glob
import json
import operator

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import keras.callbacks
from keras.optimizers import SGD

import coremltools

from PIL import Image  

Using TensorFlow backend.


In [2]:
weights = 'disdat-120-v1-softmax-lr0.001-decay0'
label_file='labels.txt'
data_folder='../data'
output='disdatkerasv8'
num_classes=120

write_labels=True
load_only=False

In [3]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

In [4]:
input_shape=(7,7,512)

In [5]:
if write_labels:
    datagen = ImageDataGenerator(rescale=1. / 255)
    data_generator = datagen.flow_from_directory(
        data_folder,
        target_size=(150, 150),
        batch_size=11,
        class_mode='sparse'
    )

    class_dictionary = data_generator.class_indices

    sorted_predictions = sorted(class_dictionary.items(), key=operator.itemgetter(1))
    labels=[p[0] for p in sorted_predictions]

    with open(label_file, 'w') as f:
        for label in labels:
            f.write(label+"\n")

Found 116228 images belonging to 120 classes.


In [8]:
with open('configs/'+weights+'.config') as config_file:    
    config=json.load(config_file)

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))
decay=1/100 * (config['lr'])/(100000/config['batch_size'])
top_model.compile(optimizer=SGD(lr=config['lr'], decay=config['decay']),
              loss='categorical_crossentropy', metrics=['accuracy'])

top_model.load_weights(weights+'.h5')

In [9]:
model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

Save the model

In [10]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
if load_only:
    coreml_model = coremltools.models.MLModel(output+'top.mlmodel')
else:
    coreml_model  = coremltools.converters.keras.convert(model,
                                                        image_input_names='image',
                                                        class_labels=label_file,
                                                        input_names=['image'],  
    #                                                     red_bias=-123.682/255,
    #                                                     green_bias=-116.78/255,
    #                                                     blue_bias=-103.94/255,
                                                        image_scale=1/255.0
    )
    
# Set model metadata
coreml_model.author = 'Balloon Inc. VOF'
coreml_model.short_description = 'Custom object detection for 120 simple classes.'


coreml_model.save(output+'top.mlmodel')

0 : input_1, <keras.engine.topology.InputLayer object at 0x7f0ca51f0b50>
1 : block1_conv1, <keras.layers.convolutional.Conv2D object at 0x7f0ca51f0b90>
2 : block1_conv1__activation__, <keras.layers.core.Activation object at 0x7f0c6076bd10>
3 : block1_conv2, <keras.layers.convolutional.Conv2D object at 0x7f0ca51f0c10>
4 : block1_conv2__activation__, <keras.layers.core.Activation object at 0x7f0c6076bd50>
5 : block1_pool, <keras.layers.pooling.MaxPooling2D object at 0x7f0c65974fd0>
6 : block2_conv1, <keras.layers.convolutional.Conv2D object at 0x7f0c659cc910>
7 : block2_conv1__activation__, <keras.layers.core.Activation object at 0x7f0c6076bbd0>
8 : block2_conv2, <keras.layers.convolutional.Conv2D object at 0x7f0c6598ea50>
9 : block2_conv2__activation__, <keras.layers.core.Activation object at 0x7f0c6076be50>
10 : block2_pool, <keras.layers.pooling.MaxPooling2D object at 0x7f0c659a8f10>
11 : block3_conv1, <keras.layers.convolutional.Conv2D object at 0x7f0c65950fd0>
12 : block3_conv1__act

## Full model weights

In [12]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))

# add the model on top of the convolutional base
refined_model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

refined_model.load_weights(weights+'-refined.h5')
refined_model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=2e-4, momentum=0.9),
              metrics=['accuracy'])

In [13]:
if load_only:
    coreml_model_refined = coremltools.models.MLModel(output+'.mlmodel')
else:
    coreml_model_refined  = coremltools.converters.keras.convert(refined_model,
                                                        image_input_names='image',
                                                        class_labels=label_file,
                                                        input_names=['image'],  
    #                                                     red_bias=-123.68/255,
    #                                                     green_bias=-116.78/255,
    #                                                     blue_bias=-103.94/255,
                                                        image_scale=1/255.
    )

0 : input_2, <keras.engine.topology.InputLayer object at 0x7f0c64624810>
1 : block1_conv1, <keras.layers.convolutional.Conv2D object at 0x7f0c607eadd0>
2 : block1_conv1__activation__, <keras.layers.core.Activation object at 0x7f0c6076bc50>
3 : block1_conv2, <keras.layers.convolutional.Conv2D object at 0x7f0c64624850>
4 : block1_conv2__activation__, <keras.layers.core.Activation object at 0x7f0c6076bdd0>
5 : block1_pool, <keras.layers.pooling.MaxPooling2D object at 0x7f0c646248d0>
6 : block2_conv1, <keras.layers.convolutional.Conv2D object at 0x7f0c645d7ed0>
7 : block2_conv1__activation__, <keras.layers.core.Activation object at 0x7f0c6076bd90>
8 : block2_conv2, <keras.layers.convolutional.Conv2D object at 0x7f0c6456dc50>
9 : block2_conv2__activation__, <keras.layers.core.Activation object at 0x7f0c6076bd50>
10 : block2_pool, <keras.layers.pooling.MaxPooling2D object at 0x7f0c6457ee50>
11 : block3_conv1, <keras.layers.convolutional.Conv2D object at 0x7f0c64593850>
12 : block3_conv1__act

In [14]:
# Set model metadata
coreml_model_refined.author = 'Balloon Inc. VOF'
coreml_model_refined.short_description = 'Custom object detection for 120 simple classes.'

coreml_model_refined.save(output+'-refined.mlmodel')

In [ ]:
import balloonml

img_path='test_images/'
images = ['calculator.jpg',
'dog.jpg',
'cat.jpg',
'keyboard.jpg',
'macbook.jpg',
'mouse.jpg',
'banana.jpg']
for image in images:
    print('Predicting '+image)
    path = img_path+image
    # keras prediction
    print(balloonml.predict_top(path, weights+'.h5',class_dictionary))
    print(balloonml.predict_fine(model,class_dictionary, img_path=path))
    
    # CoreML prediction
    im = Image.open(path)  
    coreml_model.predict({'data': im})  
    coreml_model_refined.predict({'data': im})

## Test images for v5

Predicting calculator.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('calculator', 0.021121915), ('keyboard', 0.00038548166), ('key', 6.198656e-05), ('hairbrush', 5.6152505e-05)]
[('calculator', 0.2597464), ('keyboard', 0.00038322349), ('key', 9.7186203e-06), ('computer mouse', 2.592287e-06)]
Predicting dog.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('cat', 0.0047040172), ('dog', 0.0040679271), ('sofa', 0.00085722323), ('dinosaur', 0.00027150256)]
[('cat', 0.0052334545), ('dog', 0.0040698401), ('dinosaur', 0.00085430447), ('sofa', 0.00056111149)]
Predicting cat.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('cat', 0.29206869), ('dog', 0.0055200113), ('star', 5.3297485e-05), ('dinosaur', 3.7203841e-05)]
[('cat', 0.35923439), ('dog', 0.00079333514), ('star', 2.7750855e-05), ('dinosaur', 1.5055543e-05)]
Predicting keyboard.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('radiator', 0.0022044962), ('curtain', 0.00061480148), ('ruler', 0.0005549785), ('blackboard', 0.00050748681)]
[('ruler', 0.0010941911), ('keyboard', 0.00091195788), ('computer mouse', 0.00040513425), ('guitar', 0.00040082214)]
Predicting macbook.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('keyboard', 0.01001082), ('monitor', 0.00050369196), ('blackboard', 0.00025121815), ('computer mouse', 0.00018567205)]
[('keyboard', 0.047883309), ('monitor', 0.00047867949), ('computer mouse', 0.00031897088), ('calculator', 0.00015937244)]
Predicting mouse.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('soap', 0.0042663808), ('computer mouse', 0.00091027975), ('bowl', 0.00067775662), ('pan', 0.00041909842)]
[('soap', 0.0056782723), ('computer mouse', 0.0010733134), ('bowl', 0.00028223582), ('hat', 0.00015233312)]
Predicting banana.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('banana', 0.012315591), ('bowl', 0.00019367522), ('sock', 0.00017960947), ('carrot', 5.7873309e-05)]
[('banana', 0.043339983), ('bowl', 0.00051319768), ('onion', 0.00012041288), ('soap', 0.00011019995)]
